In [1]:
import os
os.chdir('/mnt/c/Users/moritz.osterberger/Desktop/TransGourmet/Data')
from tramer.data_prep import read_and_preprocess_data
from tramer.data_prep import create_metadata_tables
from tramer.lightFM_functions import build_lightFM_dataset
from tramer.lightFM_functions import train_light_fm_model

In [2]:
from tramer.neighbours_functions import get_similiar_user_euclidean
from tramer.neighbours_functions import create_warengruppen_vergleich
from tramer.neighbours_functions import create_warengruppen_vergleich_all
#from tramer.recommender_functions import get_bought_items

#  Read & Preprocess

In [ ]:
kundenstammdaten, artikelstammdaten, sales_data, sales_data_grouped = read_and_preprocess_data("./Dimensions/Artikelstammdaten_Foodservice.xlsx","./Dimensions/Kundenstammdaten_Foodservice.xlsx","./sales_Data_2019_Zusteller_ertrag.csv",filter_to_betrieb_54 = True)

In [ ]:
sales_data.head()

In [ ]:
sales_data_grouped.head()

# Metadata tables

In [10]:
artikel_meta, kunden_meta = create_metadata_tables(kundenstammdaten,artikelstammdaten,sales_data_grouped)

In [2]:
print(kunden_meta.head())
print(artikel_meta.head())

NameError: name 'kunden_meta' is not defined

# Light FM Dataset

No user- & item features,
Kennzahl: "Bestellt"

In [1]:
dataset, interactions, weights, item_features,user_features, mappings = build_lightFM_dataset(artikel_meta, kunden_meta, sales_data_grouped, kundenstammdaten, artikelstammdaten, weight_for_interaction="Bestellt", item_feature=None, user_feature=None)

NameError: name 'build_lightFM_dataset' is not defined

In [13]:
num_kunden, num_artikel = dataset.interactions_shape()
print("Anzahl Kunden: {}, Anzahl Artikel: {}.".format(num_kunden, num_artikel))
print("Anzahl Branchen: {}, Anzahl Warengruppen: {}.".format(len(kunden_meta["Branchenschlüssel"].unique()),len(artikel_meta["Warengruppe"].unique())))
print("Sparsity: {}%".format(round(float(sales_data_grouped.shape[0])/ float(num_kunden * num_artikel)* 100,2,)))

Anzahl Kunden: 1192, Anzahl Artikel: 20626.
Anzahl Branchen: 26, Anzahl Warengruppen: 294.
Sparsity: 1.12%


#  Light FM Model

In [14]:
hyperparam = {"learning_schedule": 'adagrad',  
               "learning_rate": 0.05, 
               "user_alpha": 1e-5,
               "item_alpha": 1e-5,
               "no_components": 30,
               "max_sampled": 41,
               "loss": 'warp'}
model = train_light_fm_model(dataset, interactions, user_features, item_features, hyperparam, 30, weights)

In [ ]:
model.get_user_representations()[1]#[1]

# Find neighbours

In [15]:
cut_off = 0.8
df_similar = get_similiar_user_euclidean(model, mappings, dataset, cut_off)
df_similar

,Hauptkundennummer,neighbour,Similarity
0,V5400051,V5416134,0.706
1,V5400051,V5414501,0.719
2,V5400051,V5415089,0.756
3,V5400051,V5415912,0.758
4,V5400051,V5415609,0.781
...,...,...,...
5875,V5494515,V5494514,0.680
5876,V5494515,V5490975,0.736
5877,V5494515,V5494511,0.791
5878,V5494517,V5494513,0.750


# Warengruppenvergleich

In [16]:
warengruppen_merged_all = create_warengruppen_vergleich_all(mappings, sales_data_grouped, interactions, df_similar, cut_off, allewarengruppen = True)
warengruppen_merged_all

/mnt/c/Users/moritz.osterberger/Desktop/TransGourmet/tramer/src/tramer/neighbours_functions.py:66: RuntimeWarning: invalid value encountered in double_scalars
  1 - cut_off)).sum()),
/mnt/c/Users/moritz.osterberger/Desktop/TransGourmet/tramer/src/tramer/neighbours_functions.py:76: RuntimeWarning: invalid value encountered in double_scalars
  1 - cut_off)).sum(), 3),


,Warengruppe,Kernumsatz_Kunde,KBSpanne_Kunde,Count_Nachbarn_Warengruppe_gekauft,Kernumsatz_max,Kernumsatz_mean,Kernumsatz_weighted_mean,Kernumsatz_median,Kernumsatz_st_abwei,Kernumsatz_min,...,Spanne_max,Spanne_mean,Spanne_weighted_mean,Spanne_median,Spanne_st_abwei,Spanne_min,Spanne_3_quantile,Diff_Kernumsatz,Diff_KBSpanne,Hauptkundennummer
0,Abfallbeseit.(Säcke),237.93,1.982017,1.0,98.40,98.40,98.0,98.40,0.00,98.40,...,0.711,0.711,0.711,0.711,0.000,0.711,0.711,139.93,1.271017,V5400051
1,Allzweckreiniger,467.69,2.923959,1.0,18.86,18.86,19.0,18.86,0.00,18.86,...,0.453,0.453,0.453,0.453,0.000,0.453,0.453,448.69,2.470959,V5400051
2,Antipasti frisch,586.23,0.282705,1.0,184.05,184.05,184.0,184.05,0.00,184.05,...,0.301,0.301,0.301,0.301,0.000,0.301,0.301,402.23,-0.018295,V5400051
3,Aufschnitt,1693.43,0.690987,2.0,332.47,189.40,193.0,189.40,202.33,46.33,...,0.306,0.227,0.225,0.227,0.112,0.148,0.266,1500.43,0.465987,V5400051
4,Bad- und WC-Reiniger,250.49,0.449160,1.0,10.70,10.70,11.0,10.70,0.00,10.70,...,0.417,0.417,0.417,0.417,0.000,0.417,0.417,239.49,0.032160,V5400051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230785,Fingerfood/Snack fr.,0.00,0.000000,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000000,V5494517
230786,Küchenreiniger,0.00,0.000000,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000000,V5494517
230787,Milchmischgetränke,0.00,0.000000,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000000,V5494517
230788,Kleingebäck,0.00,0.000000,0.0,0.00,0.00,0.0,0.00,0.00,0.00,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.000000,V5494517
